# Submit functions

In this part of the tutorial, we'll learn how to change pages and questions after the user submits them.

Submit functions are like compile functions in allowing us to change a survey based on the user's responses and can often be used to accomplish the same tasks.

Run the cell below to create a test app.

In [ ]:
import os

from hemlock import User, Page, create_test_app
from hemlock.questions import Input, Label, Select
from sqlalchemy_mutable.utils import partial

os.environ.pop("GITPOD_HOST", None)

app = create_test_app()

Run the cell below to get a sense of the "page logic." When the user submits the 0th page, the following things happen:

0. The 0th page's validate functions are called.
1. If the responses are valid, the 0th page's submit functions are called.
2. The 1st page's compile functions are called.
3. The user sees the 1st page.

In [ ]:
def seed():
    return [
        Page(
            Label("Hello, world!", validate=validate_func, submit=submit_func)
        ),
        Page(
            Label("Goodbye, world!", compile=compile_func)
        )
    ]


def compile_func(parent):
    print(f"compile_func was called with parent {parent}.")


def validate_func(parent):
    print(f"validate_func was called with parent {parent}.")


def submit_func(parent):
    print(f"submit_func was called with parent {parent}.")


user = User.make_test_user(seed)
_ = user.test_request()

We can use submit functions to ask for the user's name on page 0 and greet them by name on page 1.

In [ ]:
def seed():
    pages = [
        Page(
            name_input := Input("What's your name?")
        ),
        Page(
            greet_label := Label()
        )
    ]
    name_input.submit = partial(greet_user, greet_label)
    return pages


def greet_user(name_input, greet_label):
    greet_label.label = f"Hello, {name_input.response}!"


user = User.make_test_user(seed)
user.test_request(["Sophocles"]).display()

## Exercises

0. Create a survey with 2 pages:

    0. A page with a `Select` question asking the user what their favorite planet is (you don't have to list all the planets, just 2 or 3)
    1. A goodbye page.
1. Add a submit function to the label of the goodbye page. If the user said Mars was their favorite planet, the label should change to, "No way! Mars is my favorite planet, too!" Otherwise, the label should change to, "USER'S_FAVORITE_PLANET is pretty cool, but I like Mars better."
2. With test users, verify that the survey is behaving as expected.
3. Transfer the seed function you wrote in steps 1-2 to `src/my_survey.py`, run the app, and verify that it's working as expected.
4. Test your code with `make test`.

In [ ]:
# WRITE YOUR CODE HERE

## Answers

In [ ]:
def seed():
    pages = [
        Page(
            favorite_planet := Select(
                "Which planet do you like best?",
                ["Mercury", "Jupiter", "Mars"]
            )
        ),
        Page(
            label := Label()
        )
    ]
    favorite_planet.submit = partial(display_my_favorite_planet, label)
    return pages


def display_my_favorite_planet(favorite_planet, label):
    if favorite_planet.response == "Mars":
        label.label = "No way! Mars is my favorite planet, too!"
    else:
        label.label = f"{favorite_planet.response} is pretty cool, but I like Mars better."


user = User.make_test_user(seed)
user.test_request(["Mars"]).display()

In [ ]:
user = User.make_test_user(seed)
user.test_request(["Jupiter"]).display()

See `src/submit.py` for what your survey file should look like.

Now you know how to use submit functions! Check out `090_navigate.ipynb` for the next part of the tutorial.